## Evaluación II (Parte 1)
### Evaluación de contenidos 2 Módulo 2
----

En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.

In [1]:
import requests
import pandas as pd
import numpy as np
import mysql.connector
from geopy.geocoders import Nominatim

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es: 

API_URL = "http://universities.hipolabs.com/search?country=NOMBREPAIS"

In [2]:
def llamada_API(lista_paises):
  """
Esta función crea un DataFrame con la información obtenida de la API.
Args: 
  lista_paises(list): recibe una lista con los paises que queremos consultar.
Retuns: devuelve el DataFrame con la información que queremos obtener.
  """
  # Dataframe vacío para añadir la información de cada uno de los países
  df_universidades = pd.DataFrame()

  for pais in lista_paises:
      url = f'http://universities.hipolabs.com/search?country = {pais}'
      response = requests.get(url = url)
      df_pais = pd.json_normalize(response.json())
        
      # unimos los dos DataFrames con el método .concat()
      df_universidades = pd.concat([df_universidades, df_pais], axis = 0)
    
  return df_universidades

In [3]:
# Llamamos a la función: la información de Estados Unidos no aparecía con ese nombre, aparece como United States.
df_universidades = llamada_API(['Canada', 'United States', 'Argentina'])
df_universidades.head()

,domains,country,name,web_pages,state-province,alpha_two_code
0,[cstj.qc.ca],Canada,Cégep de Saint-Jérôme,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",Quebec,CA
1,"[lambtoncollege.ca, mylambton.ca]",Canada,Lambton College,[https://www.lambtoncollege.ca],Ontario,CA
2,[acadiau.ca],Canada,Acadia University,[http://www.acadiau.ca/],Nova Scotia,CA
3,[algonquincollege.com],Canada,Algonquin College,[http://www.algonquincollege.com/],Ontario,CA
4,[ashtoncollege.com],Canada,Ashton College,[http://www.ashtoncollege.com/],British Columbia,CA


In [4]:
# Compruebo que tenemos los tres países
df_universidades['country'].value_counts()

United States    4536
Canada            300
Argentina         172
Name: country, dtype: int64

In [5]:
# Compruebo el nº de filas y columnas que tenemos en el DataFrame
df_universidades.shape

(5008, 6)

2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.
- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains

In [6]:
def limpieza_cols(dataframe):
  """
Esta función realiza una limpieza de las columnas.
Args: 
  DataFrame: DataFrame a limpiar.
Returns: None
  """
  cols_modif = {col : col.replace('-', '_') for col in dataframe.columns}

  dataframe.rename(columns = cols_modif, inplace = True)
    
  dataframe.drop(columns = 'domains', axis = 1, inplace = True)

In [7]:
# Aplicamos la función al DataFrame
limpieza_cols(df_universidades)

In [8]:
# Comprobamos si se han realizado los cambios
df_universidades.head()

,country,name,web_pages,state_province,alpha_two_code
0,Canada,Cégep de Saint-Jérôme,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",Quebec,CA
1,Canada,Lambton College,[https://www.lambtoncollege.ca],Ontario,CA
2,Canada,Acadia University,[http://www.acadiau.ca/],Nova Scotia,CA
3,Canada,Algonquin College,[http://www.algonquincollege.com/],Ontario,CA
4,Canada,Ashton College,[http://www.ashtoncollege.com/],British Columbia,CA


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. El objetivo de este ejercicio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. 

In [9]:
def explorar_cols(dataframe, columna):
  """
Esta función separa los valores de la columna indicada que está en formato lista y crea nuevas filas con esos elementos.
Args: 
  DataFrame: DataFrame sobre el que queremos trabajar.
  columna(str): columna sobre la que realizar la modificación.
Returns: devuelve el DataFrame con los valores separados de la columna indicada.
  """
  return dataframe.explode(columna)

In [10]:
# Aplicamos la función a nuestro DataFrame
df_universidades = explorar_cols(df_universidades, 'web_pages')
df_universidades

,country,name,web_pages,state_province,alpha_two_code
0,Canada,Cégep de Saint-Jérôme,https://www.cstj.qc.ca,Quebec,CA
0,Canada,Cégep de Saint-Jérôme,https://ccmt.cstj.qc.ca,Quebec,CA
0,Canada,Cégep de Saint-Jérôme,https://ccml.cstj.qc.ca,Quebec,CA
1,Canada,Lambton College,https://www.lambtoncollege.ca,Ontario,CA
2,Canada,Acadia University,http://www.acadiau.ca/,Nova Scotia,CA
...,...,...,...,...,...
167,Argentina,Universidad Nacional de Tres de Febrero,http://www.untref.edu.ar/,Buenos Aires,AR
168,Argentina,Universidad Torcuato di Tella,http://www.utdt.edu/,Ciudad Autónoma de Buenos Aires,AR
169,Argentina,Universidad Tecnológica Nacional,http://www.utn.edu.ar/,Ciudad Autónoma de Buenos Aires,AR
170,Argentina,Universidad Abierta Interamericana,http://www.vaneduc.edu.ar/uai/,Ciudad Autónoma de Buenos Aires,AR


In [11]:
# Compruebo el nº de filas y columnas que tenemos en el DataFrame
df_universidades.shape

(5034, 5)

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [12]:
# Sacamos los duplicados de la columna 'name'
df_universidades.duplicated(subset = 'name').sum()

2543

In [13]:
def eliminar_duplicados(dataframe, columna):
  """
Esta función elimina los duplicados de la columna indicada.
Args: 
  DataFrame: DataFrame sobre el que queremos trabajar.
  columna(str): columna sobre la que queremos eliminar esos duplicados.
Returns: None
  """
  dataframe.duplicated(subset = columna).sum()

  # utilizo el método drop_duplicates()
  dataframe.drop_duplicates(subset = columna, inplace = True)

In [14]:
# Aplicamos la función en el DataFrame
eliminar_duplicados(df_universidades, 'name')

In [15]:
# Compruebo que se han eliminado las filas duplicadas
df_universidades.shape

(2491, 5)

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [16]:
# Saco los unicos de la columna 'state_province' para comprobar que tenemos None
df_universidades['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', None,
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX',
       'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA', 'Illinois',
       'New Hampshire', 'North Carolina', 'South Carolina', 'Virginia',
       'Washington', 'Missouri', 'California', 'NY', 'ND', 'MI',
       'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

In [17]:
# Utilizo el método fillna para realizar la modificación
def reemplazar_none(dataframe, columna): # para todo incluimos un args y dentro del for la linea de código
  """
Esta función reemplaza los valores None por nulos de numpy.
Args: 
  DataFrame: DataFrame sobre el que queremos trabajar.
  columna(str): columna sobre la que queremos reemplazar los None.
Return: el DataFrame modificado.
  """    
  dataframe[columna].fillna(value = np.nan, inplace = True)
    
  return dataframe

In [18]:
# Aplicamos la función en el DataFrame
reemplazar_none(df_universidades, 'state_province')

,country,name,web_pages,state_province,alpha_two_code
0,Canada,Cégep de Saint-Jérôme,https://www.cstj.qc.ca,Quebec,CA
1,Canada,Lambton College,https://www.lambtoncollege.ca,Ontario,CA
2,Canada,Acadia University,http://www.acadiau.ca/,Nova Scotia,CA
3,Canada,Algonquin College,http://www.algonquincollege.com/,Ontario,CA
4,Canada,Ashton College,http://www.ashtoncollege.com/,British Columbia,CA
...,...,...,...,...,...
81,Argentina,Universidad Nacional de Tres de Febrero,http://www.untref.edu.ar/,Buenos Aires,AR
82,Argentina,Universidad Torcuato di Tella,http://www.utdt.edu/,Ciudad Autónoma de Buenos Aires,AR
83,Argentina,Universidad Tecnológica Nacional,http://www.utn.edu.ar/,Ciudad Autónoma de Buenos Aires,AR
84,Argentina,Universidad Abierta Interamericana,http://www.vaneduc.edu.ar/uai/,Ciudad Autónoma de Buenos Aires,AR


In [19]:
# Compruebo que se han cambiado los None por nan
df_universidades['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', nan,
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX',
       'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA', 'Illinois',
       'New Hampshire', 'North Carolina', 'South Carolina', 'Virginia',
       'Washington', 'Missouri', 'California', 'NY', 'ND', 'MI',
       'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [20]:
# Utilizo el método fillna para realizar la modificación
def reemplazar_nulos(dataframe, columna, valor):
  """
Esta función reemplaza los valores nulos por el valor indicado.
Args: 
  DataFrame: DataFrame sobre el que queremos trabajar.
  columna(str): columna sobre la que queremos reemplazar los nulos.
  valor(str): valor por el que queremos reemplazarlos.
Return: el DataFrame modificado.
  """      
  dataframe[columna].fillna(value = valor, inplace = True)
    
  return dataframe

In [21]:
# Aplicamos la función en el DataFrame
reemplazar_nulos(df_universidades, 'state_province', 'Unknow')

,country,name,web_pages,state_province,alpha_two_code
0,Canada,Cégep de Saint-Jérôme,https://www.cstj.qc.ca,Quebec,CA
1,Canada,Lambton College,https://www.lambtoncollege.ca,Ontario,CA
2,Canada,Acadia University,http://www.acadiau.ca/,Nova Scotia,CA
3,Canada,Algonquin College,http://www.algonquincollege.com/,Ontario,CA
4,Canada,Ashton College,http://www.ashtoncollege.com/,British Columbia,CA
...,...,...,...,...,...
81,Argentina,Universidad Nacional de Tres de Febrero,http://www.untref.edu.ar/,Buenos Aires,AR
82,Argentina,Universidad Torcuato di Tella,http://www.utdt.edu/,Ciudad Autónoma de Buenos Aires,AR
83,Argentina,Universidad Tecnológica Nacional,http://www.utn.edu.ar/,Ciudad Autónoma de Buenos Aires,AR
84,Argentina,Universidad Abierta Interamericana,http://www.vaneduc.edu.ar/uai/,Ciudad Autónoma de Buenos Aires,AR


In [22]:
# Compruebo que se han cambiado los nan por 'Unknow'
df_universidades['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknow',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX',
       'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA', 'Illinois',
       'New Hampshire', 'North Carolina', 'South Carolina', 'Virginia',
       'Washington', 'Missouri', 'California', 'NY', 'ND', 'MI',
       'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy. Para desarrollar este ejercicio deberéis:
- Sacar los valores únicos de la columna state_province.

In [23]:
# Saco los valores únicos de la columna 'state_province'
df_universidades['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknow',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA', 'TX',
       'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA', 'Illinois',
       'New Hampshire', 'North Carolina', 'South Carolina', 'Virginia',
       'Washington', 'Missouri', 'California', 'NY', 'ND', 'MI',
       'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

NV: reemplazalo por Nevada

TX: reemplazalo por Texas

IN: reemplazalo por Indianapolis

CA: reemplazalo por California

VA: reemplazalo por Virginia

NY: reemplazalo por New York

MI: reemplazalo por Michigan

GA: reemplazalo por Georgia

ND: reemplazalo por North Dakota

In [24]:
# Creo un diccionario con las siglas y los valores nuevos
dicc_provincias = {'NV': 'Nevada',
                'TX': 'Texas',
                'IN': 'Indianapolis',
                'CA': 'California',
                'VA': 'Virginia',
                'NY': 'New York',
                'MI': 'Michigan',
                'GA': 'Georgia',
                'ND': 'North Dakota'}

In [25]:
# Utilizo el método replace para realizar la modificación por los valores que tenemos en el diccionario
def reemplazar_valores(dataframe, columna, diccionario):
  """
Esta función reemplaza los valores que tenemos por los que indicamos en el diccionario.
Args: 
  DataFrame: DataFramesobre el que queremos trabajar.
  columna(str): columna sobre la que queremos reemplazar los valores.
  diccionario(dict): diccionario con los valores que queremos reemplazar.
Returns: None
  """          
  for k, v in diccionario.items():
      dataframe[columna].replace(to_replace = k, value = v, inplace = True)

In [26]:
# Aplicamos la función en el DataFrame
reemplazar_valores(df_universidades, 'state_province', dicc_provincias)

In [27]:
# Compruebo que se han modificado
df_universidades['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknow',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa', 'Virginia',
       'Indiana', 'Colorado', 'Ohio', 'Indianapolis', 'New York',
       'California', 'Illinois', 'New Hampshire', 'North Carolina',
       'South Carolina', 'Washington', 'Missouri', 'North Dakota',
       'Michigan', 'Florida', 'Georgia', 'New York, NY', 'Maine',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

- Otros valores que tenemos más formateados son y que deberemos reemplazar:

New York, NY. Deberéis reemplazarlo por "New York".

'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

In [28]:
# Creo un diccionario con los valores que tenemos y los valores nuevos
dicc_ciudades = {'Ciudad Autónoma de Buenos Aires':'Buenos Aires',
                 'New York, NY':'New York'}

In [29]:
# Aplicamos la misma función en el DataFrame que en el paso anterior
reemplazar_valores(df_universidades, 'state_province', dicc_ciudades)

In [30]:
# Compruebo que se han modificado
df_universidades['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknow',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa', 'Virginia',
       'Indiana', 'Colorado', 'Ohio', 'Indianapolis', 'New York',
       'California', 'Illinois', 'New Hampshire', 'North Carolina',
       'South Carolina', 'Washington', 'Missouri', 'North Dakota',
       'Michigan', 'Florida', 'Georgia', 'Maine', 'Buenos Aires',
       'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza', 'Santa Fé',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán'],
      dtype=object)

- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [31]:
# Creo una lista con los valores únicos de la columna 'state_province'
lista_prov_unis = list(df_universidades['state_province'].unique())
lista_prov_unis 

['Quebec',
 'Ontario',
 'Nova Scotia',
 'British Columbia',
 'Alberta',
 'Manitoba',
 'New Brunswick',
 'Saskatchewan',
 'Unknow',
 'Newfoundland and Labrador',
 'Prince Edward Island',
 'Yukon',
 'Pennsylvania',
 'Texas',
 'Utah',
 'Nevada',
 'Iowa',
 'Virginia',
 'Indiana',
 'Colorado',
 'Ohio',
 'Indianapolis',
 'New York',
 'California',
 'Illinois',
 'New Hampshire',
 'North Carolina',
 'South Carolina',
 'Washington',
 'Missouri',
 'North Dakota',
 'Michigan',
 'Florida',
 'Georgia',
 'Maine',
 'Buenos Aires',
 'Entre Ríos',
 'Salta',
 'Córdoba',
 'Mendoza',
 'Santa Fé',
 'Santiago Del Estero',
 'Misiones',
 'Catamarca',
 'Formosa',
 'Jujuy',
 'La Rioja',
 'La Pampa',
 'San Juan',
 'San Luis',
 'Tucumán']

- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [33]:
# Creo un diccionario para almacenar las listas en las que se van a introducir los resultados obtenidos
def extraer_lat_long(lista):
  """
Esta función extrae la latitud y la longuitud de cada una de las provincias de la lista indicada, obtenidas a través de la API de geopy. 
Args: 
  lista(list): lista con los valores únicos de las provincias.
Returns: el DataFrame con las dos nuevas columnas (latitud y longitud) de cada provincia.
  """ 
  dicc = {'state_province':[], 'latitud':[], 'longitud':[]}
  
  geo = Nominatim(user_agent = 'Vero')

  for provincia in lista:
    if provincia == 'Unknown':
      pass
    
    else:
      localizacion = geo.geocode(provincia)
      
      if localizacion is not None:
        dicc['state_province'].append(provincia)
        dicc['latitud'].append(localizacion[1][0])
        dicc['longitud'].append(localizacion[1][1])

  df_coordenadas = pd.DataFrame(dicc)
    
  return df_coordenadas

In [34]:
# Aplico la función para obtener las coordenadas
df_coordenadas = extraer_lat_long(lista_prov_unis)
df_coordenadas

,state_province,latitud,longitud
0,Quebec,52.476089,-71.825867
1,Ontario,50.000678,-86.000977
2,Nova Scotia,45.196040,-63.165379
3,British Columbia,55.001251,-125.002441
4,Alberta,55.001251,-115.002136
5,Manitoba,55.001251,-97.001038
6,New Brunswick,46.500283,-66.750183
7,Saskatchewan,55.532126,-106.141224
8,Unknow,8.195407,1.138812
9,Newfoundland and Labrador,53.821733,-61.229553


- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [35]:
# Utilizo el método .merge() para unir los dos DataFrames por la columna que tienen en común
df_universidades_final = df_universidades.merge(df_coordenadas, on = 'state_province', how = 'left')
df_universidades_final 

,country,name,web_pages,state_province,alpha_two_code,latitud,longitud
0,Canada,Cégep de Saint-Jérôme,https://www.cstj.qc.ca,Quebec,CA,52.476089,-71.825867
1,Canada,Lambton College,https://www.lambtoncollege.ca,Ontario,CA,50.000678,-86.000977
2,Canada,Acadia University,http://www.acadiau.ca/,Nova Scotia,CA,45.196040,-63.165379
3,Canada,Algonquin College,http://www.algonquincollege.com/,Ontario,CA,50.000678,-86.000977
4,Canada,Ashton College,http://www.ashtoncollege.com/,British Columbia,CA,55.001251,-125.002441
...,...,...,...,...,...,...,...
2486,Argentina,Universidad Nacional de Tres de Febrero,http://www.untref.edu.ar/,Buenos Aires,AR,-34.607568,-58.437089
2487,Argentina,Universidad Torcuato di Tella,http://www.utdt.edu/,Buenos Aires,AR,-34.607568,-58.437089
2488,Argentina,Universidad Tecnológica Nacional,http://www.utn.edu.ar/,Buenos Aires,AR,-34.607568,-58.437089
2489,Argentina,Universidad Abierta Interamericana,http://www.vaneduc.edu.ar/uai/,Buenos Aires,AR,-34.607568,-58.437089


In [36]:
# Compruebo el nº de filas y columnas que tenemos
df_universidades_final.shape

(2491, 7)

In [37]:
# Compruebo si tenemos alguna columna con valores nulos
df_universidades_final.isnull().sum()

country           0
name              0
web_pages         0
state_province    0
alpha_two_code    0
latitud           0
longitud          0
dtype: int64

8. Crea una BBDD en mysql que contenga las siguientes tablas:

- Tabla países: donde encontraremos las siguientes columnas:

idestado: primary key, integer, autoincremental

nombre_pais: varchar

nombre_provincia: varchar

latitud: decimal

longitud: decimal

- Tabla universidades: donde encontraremos las siguientes columnas:

iduniversidades: primary key, integer, autoincremental

nombre_universidad: varchar

pagina_web: varchar

paises_idestado: foreing key

In [38]:
def crear_bbdd(nombre_bbdd):
  """
Esta función crea una base de datos con el nombre que se indique.
Args: 
  nombre_bbdd(str): el nombre de nuestra base de datos.
Returns: None
  """
  mydb = mysql.connector.connect(host = "localhost",
                                  user = "root",
                                  password = "AlumnaAdalab") 
      
  print("Conexión realizada con éxito")

  mycursor = mydb.cursor()
      
  try:
    mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
    print(mycursor)
  
  except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

In [39]:
# Aplico la función para crear la BBDD
crear_bbdd('Universidades')

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS Universida..


In [40]:
def crear_tabla(nombre_bbdd, contraseña, query):
  """
Esta función crea una tabla en MySQL
Args: 
  nombre_bbdd(str): el nombre de nuestra base de datos.
  contraseña(str): la contraseña para conectarnos a MySQL.
  query(str): la consulta que queremos ejecutar.
Returns: None
  """
  cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    
  mycursor = cnx.cursor()
  try:
      mycursor.execute(query)
      cnx.commit()
    
  except mysql.connector.Error as err:
      print(err)
      print("Error Code:", err.errno)
      print("SQLSTATE", err.sqlstate)
      print("Message", err.msg)

In [41]:
# Primera tabla que queremos crear
query_paises = """CREATE TABLE IF NOT EXISTS `universidades`.`paises` (
                    `idestado` INT NOT NULL AUTO_INCREMENT,
                    `nombre_pais` VARCHAR(45) NULL,
                    `nombre_provincia` VARCHAR(45) NULL,
                    `latitud` DECIMAL NULL,
                    `longitud` DECIMAL NULL,
                    PRIMARY KEY (`idestado`))
                    ENGINE = InnoDB;
                    """

In [42]:
# Segunda tabla que queremos crear
query_universidades = """CREATE TABLE IF NOT EXISTS `universidades`.`universidades` (
                        `iduniversidades` INT NOT NULL AUTO_INCREMENT,
                        `nombre_universidad` VARCHAR(150) NOT NULL,
                        `pagina_web` VARCHAR(150) NULL,
                        `paises_idestado` INT NOT NULL,
                        PRIMARY KEY (`iduniversidades`),
                        CONSTRAINT `fk_universidades_paises`
                            FOREIGN KEY (`paises_idestado`)
                            REFERENCES `universidades`.`paises` (`idestado`))
                        ENGINE = InnoDB;
                        """

In [43]:
# Aplico la función para crear la primera tabla en MySQL
crear_tabla('Universidades', 'AlumnaAdalab', query_paises)

In [44]:
# Aplico la función para crear la segunda tabla en MySQL
crear_tabla('Universidades', 'AlumnaAdalab', query_universidades)